In [2]:
import pandas as pd
from textblob import TextBlob
from bs4 import BeautifulSoup
import requests
import re
import textstat
import logging

# Read the URL codes and URLs from an existing Excel file
url_data = pd.read_excel('input.xlsx')

# Replace NaN values with empty strings (or any other value you prefer)
url_data.fillna('', inplace=True)

# Extract the URL codes and URLs as separate lists
url_codes = url_data['URL_ID'].tolist()
urls = url_data['URL'].tolist()

sentiment_data = []

# Set up logging
logging.basicConfig(filename='scraping_log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

def scrape_and_analyze_sentiment(url, url_code):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception if the request was not successful (e.g., 404 or 500 error)

        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the specific <div> element that contains the <p> elements
        target_div = soup.find('div', {'class': 'td-post-content tagdiv-type'})  # Replace 'your-div-class' with the actual class name or other attributes

        if target_div:
            p_elements = target_div.find_all('p')

            sentiment_scores = [perform_sentiment_analysis(p.get_text()) for p in p_elements]

            avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
            positive_score = sum(1 for score in sentiment_scores if score > 0)
            negative_score = sum(1 for score in sentiment_scores if score < 0)

            text = " ".join([p.get_text() for p in p_elements])

            # Additional Features
            blob = TextBlob(text)
            polarity_score = blob.sentiment.polarity
            subjectivity_score = blob.sentiment.subjectivity

            avg_sentence_length = textstat.avg_sentence_length(text)

            # Calculate the percentage of complex words using the Coleman-Liau index
            complex_word_percentage = textstat.coleman_liau_index(text)

            fog_index = textstat.gunning_fog(text)
            average_word_length = textstat.avg_letter_per_word(text)
            word_count = textstat.lexicon_count(text)
            syllables_per_word = textstat.avg_syllables_per_word(text)
            personal_pronouns = len(re.findall(r'\bI\b|\bme\b|\bmy\b|\bmine\b|\bmyself\b|\bwe\b|\bus\b|\bour\b|\bours\b|\bourselves\b|\byou\b|\byour\b|\byours\b|\byourself\b|\byourselves\b|\bhe\b|\bhim\b|\bhis\b|\bhimself\b|\bshe\b|\bher\b|\bhers\b|\bherself\b|\bit\b|\bits\b|\bitself\b|\bthey\b|\bthem\b|\btheir\b|\btheirs\b|\bthemselves\b', text, re.IGNORECASE))

            sentiment_data.append({'URL Code': url_code, 'URL': url, 'Average Sentiment': avg_sentiment, 'Positive Score': positive_score, 'Negative Score': negative_score,
                                   'Polarity Score': polarity_score, 'Subjectivity Score': subjectivity_score,
                                   'Average Sentence Length': avg_sentence_length,
                                   'Percentage of Complex Words': complex_word_percentage, 'Fog Index': fog_index,
                                   'Average Word Length': average_word_length, 'Word Count': word_count,
                                   'Syllables per Word': syllables_per_word, 'Personal Pronouns': personal_pronouns,
                                   })

            logging.info(f"Processed URL: {url}")
        else:
            logging.warning(f"Could not find target div on URL: {url}")

    except requests.exceptions.RequestException as e:
        logging.error(f"Error processing URL: {url}. Error: {e}")
    except Exception as ex:
        logging.error(f"An error occurred processing URL: {url}. Error: {ex}")

# Loop through each URL and scrape content
for url, url_code in zip(urls, url_codes):
    scrape_and_analyze_sentiment(url, url_code)

# Create a Pandas DataFrame
df = pd.DataFrame(sentiment_data)

# Save DataFrame to an Excel file with UTF-8 encoding
df.to_excel('sentiment_analysis_results.xlsx', index=False, encoding='utf-8-sig')

print("Sentiment analysis results saved to 'sentiment_analysis_results.xlsx'.")

Sentiment analysis results saved to 'output.xlsx'.
